In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os
from torch.nn.utils.rnn import pad_sequence
notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break

# import whatthepatch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve,classification_report
from tqdm import tqdm as std_tqdm
from functools import partial
tqdm = partial(std_tqdm, dynamic_ncols=True)


root
ensemble_commit
/root/ensemble_commit


In [2]:
df = pd.read_csv(r'../datasets/dataset.csv', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df



/tmp/ipykernel_1757/1193077617.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,Unnamed: 0,label,message,diff
0,0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...,...
10107,10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [3]:
train, _ = train_test_split(df, train_size=0.35, random_state=42)
train, test = train_test_split(train, test_size=0.3, random_state=42)
test, val = train_test_split(test, train_size=0.5, random_state=42)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

In [4]:
# codebert_model.config.hidden_size

In [5]:
import pandas as pd
import gensim
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# 构建自定义数据集
class TextDataset(Dataset):
    def __init__(self, df, message_vocab, code_vocab, max_len):
        self.messages = df["message"].apply(lambda x: self.encode_text(x.split(), message_vocab, max_len)).tolist()
        self.codes = df["diff"].apply(lambda x: self.encode_text(x.split(), code_vocab, max_len)).tolist()
        self.labels = df["label"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        messages = np.array(self.messages[idx], dtype=np.int64)
        codes = np.array(self.codes[idx], dtype=np.int64)
        label = int(self.labels[idx])  # 确保单个标量是整数类型
        return messages, codes, label

    @staticmethod
    def encode_text(text, vocab, max_len):
        encoded = [vocab[word] if word in vocab else 0 for word in text]
        return encoded + [0] * (max_len - len(encoded))


def collate_fn(batch):
    messages, codes, labels = zip(*batch)
    
    # 确保 messages 和 codes 是二维张量，并填充
    messages = pad_sequence([torch.tensor(msg, dtype=torch.long) for msg in messages], batch_first=True, padding_value=0)
    codes = pad_sequence([torch.tensor(code, dtype=torch.long) for code in codes], batch_first=True, padding_value=0)
    
    # 确保 labels 是一维张量
    labels = torch.tensor(labels, dtype=torch.long)
    return messages, codes, labels
    
# 使用 gensim 训练 Word2Vec 模型
def train_word2vec(sentences, vector_size=100, window=5, min_count=1):
    model = gensim.models.Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count)
    return model

# 定义 LSTM + CNN 模型
class LSTM_CNN_Model(nn.Module):
    def __init__(self, message_vocab_size, code_vocab_size, embedding_dim, hidden_dim, lstm_layers, cnn_out_channels, num_classes, 
                 message_embedding_matrix=None, code_embedding_matrix=None):
        super(LSTM_CNN_Model, self).__init__()
        
        # Embedding Layers for message and code
        self.message_embedding = nn.Embedding(message_vocab_size, embedding_dim)
        self.code_embedding = nn.Embedding(code_vocab_size, embedding_dim)
        
        if message_embedding_matrix is not None:
            self.message_embedding.weight = nn.Parameter(torch.tensor(message_embedding_matrix, dtype=torch.float32))
            self.message_embedding.weight.requires_grad = False
        
        if code_embedding_matrix is not None:
            self.code_embedding.weight = nn.Parameter(torch.tensor(code_embedding_matrix, dtype=torch.float32))
            self.code_embedding.weight.requires_grad = False
        
        # LSTM Layers
        self.message_lstm = nn.LSTM(embedding_dim, hidden_dim, lstm_layers, batch_first=True, bidirectional=True)
        self.code_lstm = nn.LSTM(embedding_dim, hidden_dim, lstm_layers, batch_first=True, bidirectional=True)
        
        # CNN Layers
        self.message_conv1d = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=cnn_out_channels, kernel_size=3, stride=1, padding=1)
        self.code_conv1d = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=cnn_out_channels, kernel_size=3, stride=1, padding=1)
        
        # Fully Connected Layer
        self.fc = nn.Linear(cnn_out_channels * 2, num_classes)

    def forward(self, message, code):
        # Process message
        message_embed = self.message_embedding(message)
        message_lstm_out, _ = self.message_lstm(message_embed)
        message_lstm_out = message_lstm_out.permute(0, 2, 1)  # Change to (batch, channels, seq_len)
        message_features = torch.relu(self.message_conv1d(message_lstm_out))
        message_features = torch.mean(message_features, dim=2)  # Global Average Pooling
        
        # Process code
        code_embed = self.code_embedding(code)
        code_lstm_out, _ = self.code_lstm(code_embed)
        code_lstm_out = code_lstm_out.permute(0, 2, 1)  # Change to (batch, channels, seq_len)
        code_features = torch.relu(self.code_conv1d(code_lstm_out))
        code_features = torch.mean(code_features, dim=2)  # Global Average Pooling
        
        # Combine features
        combined_features = torch.cat([message_features, code_features], dim=1)
        output = self.fc(combined_features)
        return output



In [6]:
train

,index,Unnamed: 0,label,message,diff
0,897,897,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
1,2072,2072,0,allocator_thread: rework message sending struc...,diff --git a/src/allocator_thread.c b/src/allo...
2,8489,8489,1,...,diff --git a/coders/mpc.c b/coders/mpc.c\ninde...
3,1692,1692,0,Fix phpcs,diff --git a/src/Core/Configuration.php b/src/...
4,7905,7905,0,Merge pull request #6467 from BrickOzp/attach2...,diff --git a/Themes/default/Display.template.p...
...,...,...,...,...,...
2460,477,477,0,handles FNC1,diff --git a/lib/barby/barcode/code_128.rb b/l...
2461,7035,7035,0,Bump tox from 3.20.1 to 3.21.3\n\nBumps [tox](...,diff --git a/requirements/dev.txt b/requiremen...
2462,3539,3539,1,"Strengthen sanitization, fixes #817",diff --git a/geo-mashup-db.php b/geo-mashup-db...
2463,5333,5333,0,modify short open tag <? to <?php\n\nThe short...,diff --git a/cookieviz/settings.inc b/cookievi...


In [7]:
# 分别训练 Word2Vec 模型
message_sentences = [msg.split() for msg in train["message"]]
code_sentences = [code.split() for code in train["diff"]]
message_word2vec = train_word2vec(message_sentences, vector_size=100)
code_word2vec = train_word2vec(code_sentences, vector_size=100)

# 构建词汇表
message_vocab = {word: idx for idx, word in enumerate(message_word2vec.wv.index_to_key)}
code_vocab = {word: idx for idx, word in enumerate(code_word2vec.wv.index_to_key)}

# 构建嵌入矩阵
message_embedding_matrix = np.zeros((len(message_vocab), 100))
for word, idx in message_vocab.items():
    message_embedding_matrix[idx] = message_word2vec.wv[word]

code_embedding_matrix = np.zeros((len(code_vocab), 100))
for word, idx in code_vocab.items():
    code_embedding_matrix[idx] = code_word2vec.wv[word]

# 数据处理
max_len = 10
train_dataset = TextDataset(train, message_vocab, code_vocab, max_len)
# 创建 DataLoader 时指定 collate_fn
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [8]:
# device = torch.device('cpu')

In [9]:
test_dataset = TextDataset(test, message_vocab, code_vocab, max_len)
# 创建 DataLoader 时指定 collate_fn
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, data_loader, criterion):
    """
    Evaluate the model on the test dataset, computing loss, accuracy, precision, recall, and F1 score.
    
    Args:
        model: PyTorch model to evaluate.
        data_loader: DataLoader for the test dataset.
        criterion: Loss function to evaluate the model's performance.
        device: Device to run the evaluation (e.g., "cpu" or "cuda").
    
    Returns:
        avg_loss: Average loss on the test dataset.
        accuracy: Accuracy of the model on the test dataset.
        precision: Precision score for the test dataset.
        recall: Recall score for the test dataset.
        f1: F1 score for the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient computation for evaluation
        for messages, codes, labels in data_loader:
            # Move data to the specified device
            messages, codes, labels = torch.tensor(messages), torch.tensor(codes), torch.tensor(labels)
            
            # Forward pass
            outputs = model(messages, codes)
            loss = criterion(outputs, labels)
            
            # Update loss
            total_loss += loss.item() * labels.size(0)  # Multiply by batch size
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Collect predictions and true labels for metrics
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    
    # Compute average loss and accuracy
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    
    # Compute precision, recall, and F1 score
    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)
    print(f'Validation Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    return all_labels, all_preds


In [11]:
model  = torch.load('PatchRNN_entire_bert_model_900repo.pth')

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Evaluate the model
all_labels,all_preds  = evaluate(model, test_loader, criterion)


/tmp/ipykernel_1757/282295445.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  messages, codes, labels = torch.tensor(messages), torch.tensor(codes), torch.tensor(labels)


Validation Accuracy: 0.615530303030303
Precision: 0.6861915059144269
Recall: 0.615530303030303
F1-Score: 0.6195732204395415


In [14]:
print(classification_report(all_labels,all_preds,digits=4))

              precision    recall  f1-score   support

           0     0.8028    0.5224    0.6329       335
           1     0.4839    0.7772    0.5964       193

    accuracy                         0.6155       528
   macro avg     0.6433    0.6498    0.6147       528
weighted avg     0.6862    0.6155    0.6196       528



In [15]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent="在线模型训练完毕"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="在线模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("在线模型训练完毕")

在线模型训练完毕
